# Importare biblioteci

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# Încărcarea modelului

In [2]:
model = tf.keras.models.load_model('models\Model-final-arhitectura-mare')

# Preluarea datelor de test

In [3]:
from passwords import API_KEY, API_SECRET
from binance.client import Client
binance_api_key = API_KEY
binance_api_secret = API_SECRET
binance_client = Client(api_key=binance_api_key, api_secret=binance_api_secret)

klines1 = binance_client.get_klines(symbol='BCHUSDT', interval=Client.KLINE_INTERVAL_1HOUR)
data_orig1 = pd.DataFrame(klines1, columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
klines2 = binance_client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1HOUR)
data_orig2 = pd.DataFrame(klines2, columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
klines3 = binance_client.get_klines(symbol='ETHUSDT', interval=Client.KLINE_INTERVAL_1HOUR)
data_orig3 = pd.DataFrame(klines3, columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
klines4 = binance_client.get_klines(symbol='LTCUSDT', interval=Client.KLINE_INTERVAL_1HOUR)
data_orig4 = pd.DataFrame(klines4, columns = ['time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])


data_orig1.drop(['close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'], axis=1, inplace=True)
data_orig2.drop(['close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'], axis=1, inplace=True)
data_orig3.drop(['close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'], axis=1, inplace=True)
data_orig4.drop(['close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'], axis=1, inplace=True)


data_orig1.set_index("time", inplace=True)
data_orig1.drop(["low", "high", "open"], axis=1, inplace=True)
data_orig2.set_index("time", inplace=True)
data_orig2.drop(["low", "high", "open"], axis=1, inplace=True)
data_orig3.set_index("time", inplace=True)
data_orig3.drop(["low", "high", "open"], axis=1, inplace=True)
data_orig4.set_index("time", inplace=True)
data_orig4.drop(["low", "high", "open"], axis=1, inplace=True)

In [4]:
data1 = data_orig1.copy()
data2 = data_orig2.copy()
data3 = data_orig3.copy()
data4 = data_orig4.copy()

# Preprocesarea datelor pt. primul strat ale rețelei neuronale

In [5]:
frames = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT']
data1.rename(
        columns={"close": "BCHUSDT-close", "volume": "BCHUSDT-volume"}, inplace=True
    )
data2.rename(
        columns={"close": "BTCUSDT-close", "volume": "BTCUSDT-volume"}, inplace=True
    )
data3.rename(
        columns={"close": "ETHUSDT-close", "volume": "ETHUSDT-volume"}, inplace=True
    )
data4.rename(
        columns={"close": "LTCUSDT-close", "volume": "LTCUSDT-volume"}, inplace=True
    )

df_temp = data1
df_temp = df_temp.join(data2)
df_temp = df_temp.join(data3)
df_temp = df_temp.join(data4)

In [6]:
inference_data = df_temp.tail(16)

In [7]:
sequencial_data = []
for row in inference_data.values:
    sequencial_data.append([x for x in row])

for row in sequencial_data:
    row.append(0)

d = np.asarray(sequencial_data, dtype=np.float32)
df = np.expand_dims(d, 0)
df.shape

(1, 16, 9)

In [8]:
predictions = model.predict(df)
predictions

array([[0.65420306, 0.34579694]], dtype=float32)

# Rezultatul final

In [9]:
maxIndex = np.argmax(predictions)

pred = 'crește' if maxIndex == 1 else 'scădea'

print(f'prețul va {pred} după {1*4} ore')

prețul va scădea după 4 ore
